# Chain
![Chain](images/chain.png)

## Basic Graph
![Graph](images/graph.png)

### State
A State shows the graph’s current setup and tracks changes over time. It serves as input and output for all Nodes and Edges.

In [6]:
from typing_extensions import TypedDict

class State(TypedDict):
    state: str

### Node
a Node represents an individual element or point within the graph, storing data and connecting with other nodes through edges to form relationships.

a Node is a Python function, the first arg of the function is a State

In [3]:
def node_1(state):
    print("Node 1")
    return {"state": state["state"] + "-1-"}

def node_2(state):
    print("Node 2")
    return {"state": state["state"] + "-2-"}

def node_3(state):
    print("Node 3")
    return {"state": state["state"] + "-3-"}

### Edges
an Edge represents a connection between two nodes, defining the relationship or interaction between them and potentially carrying additional data.
types:
    - normal edges: always go this way (from node_1 to node_2)
    - conditional edges: optional route between nodes. a Pythin function that returns a next node based on a logic

In [8]:
import random
from typing import Literal

def get_random_node(state) -> Literal["node_2", "node_3"]:
    current_state = state['state'] # usually the desiction is based on current state
    if random.random() < 0.5:
        return "node_2"
    return "node_3"

### Graph Construction and Invocation
START and END are special nodes that represent the start and end of the graph.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# generate
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", get_random_node)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# building
graph = builder.compile()

# visualize
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"state" : "Hi, there!"})

## Some LLM related concepts

### Messages
chat models operate on messages.
various message types (check out our other Spring AI video):

    - HumanMessage
    - AIMessage
    - SystemMessage
    - ToolMessage (will check later)

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Initial SystemMessage to set context
messages = [
    SystemMessage(content="You are a witty and humorous AI assistant for developers, specializing in automating their daily coding headaches."),
]

messages.append(AIMessage(content="Hey there! I’m your AI assistant. Ready to debug your code—or your life?", name="Model"))
messages.append(HumanMessage(content="Can you handle all my TODO comments?", name="Dev"))
messages.append(AIMessage(content="Sure! I’ll replace 'TODO: Refactor' with 'TODO: Blame someone else.' Problem solved!", name="Model"))
messages.append(HumanMessage(content="Nice. Can you optimize my SQL too?", name="Dev"))

for message in messages:
    message.pretty_print()

### Chat models
processes messages as prompts and response with completion.
- OpenAI

In [ ]:
from langchain_openai import ChatOpenAI

# OPENAI_API_KEY environment variable must be set
llm = ChatOpenAI(model="gpt-4o-mini")
result = llm.invoke(messages)

type(result)

In [ ]:
import json

print(json.dumps(vars(result), indent=4))

### Tools
Tools help an AI use special apps or systems to do things it can’t do on its own, like checking the weather or solving tricky problems.

![Graph](images/tools.png)

In [6]:
def multiply_values(a, b):
    """
    Multiply two values and return the result.

    Parameters:
        a (float): The first value.
        b (float): The second value.

    Returns:
        float: The product of a and b.
    """
    return a * b

llm_tools = llm.bind_tools([multiply_values])

#### How does LLM know which tool to use?
- the name of the function
- docstring definition
- number of arguments
- ...

In [ ]:
tool_call = llm_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Dev")])

print(json.dumps(vars(tool_call), indent=4))

### Merging Messages with State
with LLM we're ineterested in passing messages between nodes. So they become a part of the state.

In [10]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

the problem with this approach - override of the state.
so we need to append messages to the list
we will use reducers for changing the way how state is being updated.

```python
def node_1(state):
    print("Node 1")
    return {"state": state["state"] + "-1-"}

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


initial_messages = [SystemMessage(content="You are a witty and humorous AI assistant for developers, specializing in automating their daily coding headaches.")]
new_message = AIMessage(content="Hey there! I’m your AI assistant. Ready to debug your code—or your life?", name="Model")
add_messages(initial_messages , new_message)

In [ ]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Extend to include additional keys beyond the pre-built messages key
    pass

## Combine all together
![Graph](images/tools.png)

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
    
# Node
def llm_with_tools(state: MessagesState):
    return {"messages": [llm_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("llm_with_tools", llm_with_tools)
builder.add_edge(START, "llm_with_tools")
builder.add_edge("llm_with_tools", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="How are you?")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
    m.pretty_print()